In [10]:
import pandas as pd

In [67]:
# Load datasets
customers = pd.read_csv("datasets/olist_customers_dataset.csv");
order_items = pd.read_csv("datasets/olist_order_items_dataset.csv");
order_payments = pd.read_csv("datasets/olist_order_payments_dataset.csv");
order_reviews = pd.read_csv("datasets/olist_order_reviews_dataset.csv");
orders = pd.read_csv("datasets/olist_orders_dataset.csv");
products = pd.read_csv("datasets/olist_products_dataset.csv");
sellers = pd.read_csv("datasets/olist_sellers_dataset.csv");

In [45]:
def drop_duplicates_by_column(df, column_name):
    """
    Drops all rows that have duplicate values in the specified column.
    Keeps only the first occurrence of each unique value in the column.
    
    Args:
        df (pandas.DataFrame): The DataFrame from which duplicates will be removed.
        column_name (str): The column name to check for duplicates.
        
    Returns:
        pandas.DataFrame: The cleaned DataFrame with duplicates removed.
    """
    before = df.shape[0]  # Rows before dropping duplicates
    if (column_name != ""):
        df = df.drop_duplicates(subset=[column_name], keep='first') # Remove rows with the same primary key
    df = df.drop_duplicates() # Remove duplicate rows for all columns combined
    after = df.shape[0]  # Rows after dropping duplicates
    print(f"Removed {before - after} duplicates from '{column_name}' column.")
    return df

In [56]:
print('Customers before: ' + str(customers.shape))
customers = drop_duplicates_by_column(customers, 'customer_unique_id');
customers = customers.dropna(subset=['customer_unique_id', 'customer_city', 'customer_state', 'customer_zip_code_prefix'])
print('Customers after: ' + str(customers.shape))

Customers before: (96096, 5)
Removed 0 duplicates from 'customer_unique_id' column.
Customers after: (96096, 5)


In [111]:
print('Products before: ' + str(products.shape))

products = drop_duplicates_by_column(products, 'product_id');

# Remove products without a category or name.
products['product_category_name'] = products['product_category_name'].fillna('No Category')
products['product_name_lenght'] = products['product_name_lenght'].fillna(0)

print('Products after: ' + str(products.shape))

Products before: (32340, 9)
Removed 0 duplicates from 'product_id' column.
Products after: (32340, 9)


In [114]:
sellers = drop_duplicates_by_column(sellers, 'seller_id');
sellers['seller_city'] = sellers['seller_city'].fillna('N/A')
sellers['seller_zip_code_prefix'] = sellers['seller_zip_code_prefix'].fillna('0000')

Removed 0 duplicates from 'seller_id' column.


In [ ]:
order_payments = drop_duplicates_by_column(order_payments, '');

In [ ]:
order_items = drop_duplicates_by_column(order_items, '');

In [71]:
order_reviews = drop_duplicates_by_column(order_reviews, 'review_id');
order_reviews = order_reviews.dropna(subset=['review_score']);
order_reviews['review_comment_title'] = order_reviews['review_comment_title'].fillna('No Title Available')
order_reviews['review_comment_message'] = order_reviews['review_comment_message'].fillna('No Comment Available')

Removed 814 duplicates from 'review_id' column.


In [62]:
print('Orders before: ' + str(orders.shape))

orders = drop_duplicates_by_column(orders, 'order_id');

# Since we need to calculate the Delivery Time, we need to remove rows missing the following columns: Order Delivered Customer Date, Order Delivered Carrier Date, Order Approved At

orders = orders.dropna(subset=['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date'])
print('Orders after: ' + str(orders.shape))

Orders before: (96461, 8)
Removed 0 duplicates from 'order_id' column.
Orders after: (96461, 8)


In [ ]:
# 4.2 Creating Calculated Columns

In [101]:
order_items_with_total = order_items.copy()
order_items_with_total['Total Price'] = order_items_with_total['price'] + order_items_with_total['freight_value']

print(order_items_with_total.head())

                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   
3  7634da152a4610f1595efa32f14722fc  9d7a1d34a5052409006425275ba1c2b4   
4  ac6c3623068f30de03045865e4e10089  df560393f3a51e74553ab94004ba5c87   

   shipping_limit_date   price  freight_value  profit_margin  Total Price  
0  2017-09-19 09:45:35   58.90          13.29          45.61        72.19  
1  2017-05-03 11:05:13  239.90          19.93         219.97       259.83  
2  2018-

In [102]:
# Create a copy of the orders DataFrame to avoid modifying the original
orders_with_delivery_time = orders.copy()

# Ensure both columns are in datetime format in the new DataFrame
orders_with_delivery_time['order_purchase_timestamp'] = pd.to_datetime(orders_with_delivery_time['order_purchase_timestamp'])
orders_with_delivery_time['order_delivered_customer_date'] = pd.to_datetime(orders_with_delivery_time['order_delivered_customer_date'])

# Calculate the Delivery Time as the difference between the delivered date and the purchase date
orders_with_delivery_time['Delivery Time'] = (orders_with_delivery_time['order_delivered_customer_date'] - orders_with_delivery_time['order_purchase_timestamp']).dt.days

print(orders_with_delivery_time.head())

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00           2018-08

In [79]:
# Group by 'order_id' and sum the 'payment_installments' for each order
payment_count = order_payments.groupby('order_id', as_index=False)['payment_installments'].sum()

# Rename column for clarity
payment_count.rename(columns={'payment_installments': 'total_payment_installments'}, inplace=True)

print(payment_count.head())


                           order_id  total_payment_installments
0  00010242fe8c5a6d1ba2dd792cb16214                           2
1  00018f77f2f0320c557190d7a144bdd3                           3
2  000229ec398224ef6ca0657da4fc703e                           5
3  00024acbcdf0a6daa1e931b038114c75                           2
4  00042b26cf59d7ce69dfabb4e55b4fd9                           3


In [80]:
order_items['profit_margin'] = order_items['price'] - order_items['freight_value']

print(order_items.head())

                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   
3  7634da152a4610f1595efa32f14722fc  9d7a1d34a5052409006425275ba1c2b4   
4  ac6c3623068f30de03045865e4e10089  df560393f3a51e74553ab94004ba5c87   

   shipping_limit_date   price  freight_value  profit_margin  
0  2017-09-19 09:45:35   58.90          13.29          45.61  
1  2017-05-03 11:05:13  239.90          19.93         219.97  
2  2018-01-18 14:48:30  199.00          17.87  

In [91]:
# 4.3: Total sales per customer, total of product prices for each customers orders
# Aggregate order_items at the order_id level
order_totals = order_items.groupby('order_id', as_index=False)['price'].sum()

# Merge with orders to get customer_id
merged = orders.merge(order_totals, on='order_id')

# Compute the cumulative total per customer
merged['total_sales_per_customer'] = merged.groupby('customer_id')['price'].cumsum()

# Display result
print(merged)

# Index no: 98664: order_id 11c177c8e97725db2631073c19f07b62 has 2 orders with each price being 179.99, cumulative sum is shown 359.98

                               order_id                       customer_id  \
0      e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1      53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2      47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3      949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4      ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   
...                                 ...                               ...   
98661  9c5dedf39a927c1b2549525ed64a053c  39bd1228ee8140590ac3aca26f2dfe00   
98662  63943bddc261676b46f01ca7ac2f7bd8  1fca14ff2861355f6e5f14306ff977a7   
98663  83c1379a015df1e13d02aae0204711ab  1aa71eb042121263aafbe80c1b562c9c   
98664  11c177c8e97725db2631073c19f07b62  b331b74b18dc79bcdf6532d51e1637c1   
98665  66dea50a8b16d9b4dee7af250b4be1a5  edb027a75a1449115f6b43211ae02a24   

      order_status order_purchase_timestamp    order_approved_at  \
0      

In [93]:
# Average Delivery Time from the moment a customer places an order to when it is delivered to their house
# Convert timestamps to datetime format
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])

# Calculate Delivery Time
orders['delivery_time_days'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days

# Merge all datasets to get category
merged = order_items.merge(orders[['order_id', 'delivery_time_days']], on='order_id').merge(products, on='product_id')

# Compute rolling average of delivery time per product category
merged['rolling_avg_delivery_time'] = merged.groupby('product_category_name')['delivery_time_days'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)

print(merged)

                                order_id  order_item_id  \
0       00010242fe8c5a6d1ba2dd792cb16214              1   
1       130898c0987d1801452a8ed92a670612              1   
2       532ed5e14e24ae1f0d735b91524b98b9              1   
3       6f8c31653edb8c83e1a739408b5ff750              1   
4       7d19f4ef4d04461989632411b7e588b9              1   
...                                  ...            ...   
111041  ffebd80e3291e811c308365936897efd              1   
111042  ffee31fb4b5e35c9123608015637c495              1   
111043  fff7c4452f050315db1b3f24d9df5fcd              1   
111044  fffa82886406ccf10c7b4e35c4ff2788              1   
111045  fffe41c64501cc87c801fd61db3f6244              1   

                              product_id                         seller_id  \
0       4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1       4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
2       4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b

In [109]:
# 4.4 Creating a fact table
# 1. Total Price: price + freight_value
order_items['total_price'] = order_items['price'] + order_items['freight_value']

# 2. Calculate Delivery Time
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['delivery_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days


# 3. Payment Count (Total Payment Installments for each order)
payment_count = order_payments.groupby('order_id', as_index=False)['payment_installments'].sum()
payment_count.rename(columns={'payment_installments': 'total_payment_installments'}, inplace=True)

# Merge order_items with orders and payment_count
fact_table = order_items.copy()
fact_table = fact_table.merge(orders[['order_id', 'delivery_time', 'customer_id']], on='order_id', how='left') \
                .merge(payment_count[['order_id', 'total_payment_installments']], on='order_id', how='left')

# 4. Profit Margin: price - freight_value
fact_table['profit_margin'] = order_items.groupby('order_id')['price'].transform('sum') - order_items.groupby('order_id')['freight_value'].transform('sum')

# Now, 'fact_table' contains only the `order_id` and the four calculated columns.
#fact_table = fact_table[['order_id', 'Total Price', 'Delivery Time', 'total_payment_installments', 'Profit Margin']]

# Display the final fact table
print(fact_table.head())

                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   
3  7634da152a4610f1595efa32f14722fc  9d7a1d34a5052409006425275ba1c2b4   
4  ac6c3623068f30de03045865e4e10089  df560393f3a51e74553ab94004ba5c87   

   shipping_limit_date   price  freight_value  profit_margin  Total Price  \
0  2017-09-19 09:45:35   58.90          13.29          45.61        72.19   
1  2017-05-03 11:05:13  239.90          19.93         219.97       259.83   
2  20

In [97]:
# The only dimension table needed is for Date
date_dim = pd.DataFrame();
# Keep the full timestamp for both order purchase and delivery date
date_dim['order_id'] = orders['order_id']
date_dim['order_purchase_timestamp'] = orders['order_purchase_timestamp']
date_dim['order_delivered_customer_date'] = orders['order_delivered_customer_date']

# Display the Date Dimension table
print(date_dim.head())

                           order_id order_purchase_timestamp  \
0  e481f51cbdc54678b7cc49136f2d6af7      2017-10-02 10:56:33   
1  53cdb2fc8bc7dce0b6741e2150273451      2018-07-24 20:41:37   
2  47770eb9100c2d0c44946d9cf07ec65d      2018-08-08 08:38:49   
3  949d5b44dbf5de918fe9c16f97b45f8a      2017-11-18 19:28:06   
4  ad21c59c0840e6cb83a9ceb5573f8159      2018-02-13 21:18:39   

  order_delivered_customer_date  
0           2017-10-10 21:25:13  
1           2018-08-07 15:27:45  
2           2018-08-17 18:06:29  
3           2017-12-02 00:28:42  
4           2018-02-16 18:17:02  


In [106]:
from sqlalchemy import create_engine

server = 'DESKTOP-O24039G' 
database = 'de'
username = ''
password = ''

connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)

# Save DataFrames to SQL Server
customers.to_sql('customers', con=engine, if_exists='replace', index=False)
products.to_sql('products', con=engine, if_exists='replace', index=False)
sellers.to_sql('sellers', con=engine, if_exists='replace', index=False)
date_dim.to_sql('date', con=engine, if_exists='replace', index=False)
fact_table.to_sql('orders', con=engine, if_exists='replace', index=False)


print("DataFrames saved to SQL Server successfully.")

DataFrames saved to SQL Server successfully.


-1